In [1]:
import pyautogui as pyt
import cv2 as cv
import numpy as np
import time
from tkinter import *

In [2]:
def ScreenSize():
    img = pyt.screenshot()
    img = np.array(img)
    height, width, layers = img.shape
    size = (width,height)
    return size

def Screen_Recorder(out_video, start_rec, stop_rec):
    Screen_Gui(start_rec, stop_rec)
    if start_rec=="start":
        while True:
            img = pyt.screenshot()
            frame = np.array(img)
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            cv.imshow('screenshot',frame)
            out_video.write(frame)
            if cv.waitKey(1)==stop_rec:
                break
        cv.destroyAllWindows()
        return out_video.release()
    else:
        exit(0)

In [3]:
def start_rec(event):
    start_rec = "start"
def stop_rec(event):
    stop_rec = ord("q")

In [4]:
def Screen_Gui(start_rec, stop_rec):
    src = Tk()
    src.title("Screen Recording")
    src.geometry("200x70")
    start_rec = Button(src, text = 'Start', command = start_rec)
    stop_rec = Button(src, text = 'Stop', command = stop_rec)
    start_rec.grid(row = 0, column = 0, padx = 5, pady = 5)
    stop_rec.grid(row = 0, column = 1, padx = 5, pady = 5)
    start_rec.bind('<Button-1>', start_rec)
    stop_rec.bind('<Button-1>', stop_rec)
    return src.mainloop()

In [5]:
if __name__=="__main__":
#     start_rec = str(input())
#     stop_rec = ord("q")
#     play_rec = ord("p")
#     pause_rec = ord("p")
    fourcc = cv.VideoWriter_fourcc(*"XVID")
    out_video = cv.VideoWriter("Recording.avi", fourcc, 24.0, ScreenSize())
    Screen_Recorder(out_video, start_rec(event), stop_rec(event))

NameError: name 'event' is not defined

In [ ]:
import numpy as np
import cv2
from PIL import ImageGrab
import pyautogui
from playsound import playsound
# import pyaudio
# import wave
from tkinter import *
import threading
import datetime

target = 'screen'
is_recorded = False
show_preview = True

seconds = 0
minutes = 0
hours = 0

def record_screen():
  global show_preview, is_recorded
  name = 'screen'
  now = datetime.datetime.now()
  date = now.strftime("%H%M%S")
  file_format = 'avi'
  filename = name + str(date) + '.' + file_format
  fps = 24
  resolution = (1366, 768)
  thumb_resolution = (342, 192)

  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  writer = cv2.VideoWriter(filename, fourcc, fps, resolution)

  while True:
    img = ImageGrab.grab()
    img_np = np.array(img)
    frame = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)
    writer.write(frame)
    if show_preview:
      thumb = cv2.resize(frame, dsize=thumb_resolution)
      cv2.imshow('Preview - Screen Recorder', thumb)
    if cv2.waitKey(1) == 27:
      is_recorded = False
      msg_label['text'] = 'Video was saved as ' + filename
      update_rec_btn()
      break

  writer.release()
  cv2.destroyAllWindows()

col_width = 14
bg = 'black'
clicked_bg = '#222'
font_color = 'white'
border = 0

window = Tk()
window.title('Screen Recorder')
window.geometry('360x100')
window.resizable(width=False, height=False)
window.configure(bg=bg)

frame = Frame(window)
frame.configure(bg=bg)
frame.pack()

rec_btn = Button(frame, text='REC', command=start_recording,
                 width=col_width, bg='#e50914', fg=font_color, bd=border,
                 activebackground='#ab070f', activeforeground=font_color)
rec_btn.grid(row=1, column=2)

counter_label = Label(frame, text='0:0:0',
                      width=col_width, bg=bg, fg=font_color)
counter_label.grid(row=1, column=3)

preview_btn = Button(frame, text="Preview: ON", command=update_show_preview,
                     width=col_width, bg=bg, fg=font_color, bd=border,
                     activebackground=clicked_bg, activeforeground=font_color)
preview_btn.grid(row=2, column=1)

shot_btn = Button(frame, text='Take screenshot', command=take_screenshot,
                  width=col_width, bg=bg, fg=font_color, bd=border,
                  activebackground=clicked_bg, activeforeground=font_color)
shot_btn.grid(row=2, column=3)

msg_frame = Frame(window)
msg_frame.configure(bg=bg)
msg_frame.pack()

msg_label = Label(msg_frame, width=3 * col_width, bg=bg, fg=font_color)
msg_label.pack()

window.mainloop()

def reset_time():
  global seconds, minutes, hours
  seconds = 0
  minutes = 0
  hours = 0

def display_time():
  global is_recorded, seconds, minutes, hours
  if is_recorded:
    if seconds == 60:
      seconds = 0
      minutes += 1
    elif minutes == 60:
      minutes = 0
      hours += 1

    counter_label.config(text=str(hours) + ':' + str(minutes) + ':' + str(seconds))
    seconds += 1
    counter_label.after(1000, display_time)